## Import necessary libraries

In [1]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate
from langchain import hub

from env import OPENAI_API_KEY

## Load the data

Perform data cleaning

In [2]:
loader = CSVLoader(file_path="./output.csv")
data = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

In [4]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))
retriever = vectorstore.as_retriever()

In [5]:
rag_prompt = hub.pull("rlm/rag-prompt")

In [6]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

In [7]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

### Question 1

In [8]:
rag_chain.invoke("""What are the customers in Retail industry that chose salesforce?
                 Give the answer in form of bullet points for easy grasp.""")

AIMessage(content='- Currys, a UK retailer, chose Salesforce to help create lifetime value for customers and augment the shopping experience.\n- Aldo, a retail company, uses Salesforce to bring customer data to mobile devices for better service and to inform call-center staffers for faster call resolution.\nThanks for asking!', additional_kwargs={}, example=False)

### Question 2

In [9]:
rag_chain.invoke("""Can you tell me more about customer Kellogg and how they benefited from salesforce?""")

AIMessage(content='Kellogg has benefited from Salesforce by using data to suggest products for smaller customers, reaching consumers directly, and caring for the community through donated meals and volunteer opportunities. Thanks for asking!', additional_kwargs={}, example=False)